In [ ]:
#data
!wget http://jana-ubytovani.cz/cars2min.zip

In [ ]:
import zipfile
with zipfile.ZipFile("cars2min.zip", 'r') as zip_ref:
  zip_ref.extractall("data")

In [ ]:
! pip install -q pytorch-lightning
! pip install -q pytorch-lightning-bolts

In [ ]:
import torch
import torch.nn as nn
from torch.nn import functional as F
from torch.utils.data import DataLoader, random_split
import pytorch_lightning as pl
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

! pip install -q pretrainedmodels
import pretrainedmodels
# used CNN architecture
model_name = 'inceptionresnetv2'

In [ ]:
mean = np.array([0.485, 0.456, 0.406])
std = np.array([0.229, 0.224, 0.225])
#Test if imgs are loaded, and how they look
def imgshow(inp,mean=None,std=None,title=None):
  inp = inp.numpy().transpose((1, 2, 0))

  inp = std * inp + mean
  inp = np.clip(inp, 0, 1)

  plt.imshow(inp)
  if title is not None:
    plt.title(title)
  plt.pause(0.001)

In [ ]:
#data transformations for learning and testing
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(299),
        transforms.RandomHorizontalFlip(),
        transforms.RandomVerticalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(mean, std)
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(299),
        transforms.ToTensor(),
        transforms.Normalize(mean, std)
    ]),
}

In [ ]:
#Show classes and len
dataset = datasets.ImageFolder("data/cars")
print(dataset.classes)
print(len(dataset))
type(dataset)


In [ ]:
#Split data to train and validation
train, val = random_split(dataset, [3431, 858])

train.dataset.transform = data_transforms['train']
val.dataset.transform = data_transforms['val']
#batchSize for learning
batchSize = 8

dataset.classes[0]
class_counts = torch.tensor([1280,1201, 504, 446])
numDataPoints = class_counts.sum()

target = torch.cat((torch.zeros(class_counts[0], dtype=torch.long),
                    torch.ones(class_counts[1], dtype=torch.long),
                    torch.ones(class_counts[2], dtype=torch.long)*2,
                    torch.ones(class_counts[3], dtype=torch.long)*3))

print('target train 0/1/2/3: {}/{}/{}/{}'.format(
    (target == 0).sum(), (target == 1).sum(), (target == 2).sum(),(target == 3).sum()))

# Compute samples weight (each sample should get its own weight)
class_sample_count = torch.tensor(
    [(target == t).sum() for t in torch.unique(target, sorted=True)])
weight = 1. / class_sample_count.float()
samples_weight = torch.tensor([weight[t] for t in target])
print(samples_weight)
# Create sampler, dataset, loader
sampler = torch.utils.data.sampler.WeightedRandomSampler(samples_weight, len(samples_weight))

print(val)

#create dataloaders
train_dataloader = DataLoader(train, batch_size=batchSize, num_workers=4, sampler = sampler)
val_dataloader = DataLoader(val, batch_size=batchSize, num_workers=4)


In [ ]:
class_names = dataset.classes
inputs, classes = next(iter(train_dataloader))
out = torchvision.utils.make_grid(inputs)
imgshow(out,mean,std, title=[class_names[x] for x in classes])

In [ ]:
# class for transfer learning
class CarTypeRecognizer(pl.LightningModule):

    def __init__(self, num_target_classes):
        super().__init__()
        self.model = pretrainedmodels.__dict__[model_name](num_classes=1000, pretrained='imagenet')
        num_ftrs = self.model.last_linear.in_features

        # self.model = torchvision.models.resnet50(pretrained=True)
        # num_ftrs = self.model.fc.in_features
        self.model.fc = nn.Linear(num_ftrs, num_target_classes)
        self.acc = pl.metrics.Accuracy()

    def forward(self, x):
        return self.model(x)

    def training_step(self, batch, batch_idx):
        # --------------------------
        x, y = batch

        y_hat = self(x)
        
        loss = F.cross_entropy(y_hat, y)
        self.log('train_loss', loss)
        return loss
        # --------------------------

    def training_epoch_end(self, outs):
        acc = self.acc.compute()
  
    def validation_step(self, batch, batch_idx):
        # --------------------------
        x, y = batch
        y_hat = self(x)
        loss = F.cross_entropy(y_hat, y)
        self.log('val_loss', loss)
        self.log('test_acc_step', self.acc(y_hat, y), on_step=True, on_epoch=False)
        # --------------------------
    def validation_epoch_end(self,outs):
        acc = self.acc.compute()
        self.log("test_acc_epoch", acc)   
    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=1e-4,weight_decay=1e-5)
        return optimizer

In [ ]:
#Model training
model = CarTypeRecognizer(len(class_names))
trainer = pl.Trainer(gpus=1, max_epochs=25,progress_bar_refresh_rate=20)
trainer.fit(model, train_dataloader=train_dataloader, val_dataloaders=val_dataloader)

In [ ]:
trainer.save_checkpoint('/content/data/modevSamplerMinb8_25f.ckpt');

In [ ]:
torch.save(model.state_dict(), '/content/data/modevSamplerMinb8_25f.pth')

In [ ]:
%load_ext tensorboard
%tensorboard --logdir lightning_logs/

In [ ]:
#Test data with visualize / from SMAP Cvičení 29.10. - transfer learning
import random
device = torch.device("cuda")
def vizualize_model(model, num_images=6):
  model.eval()
  images_so_far = 0
  fig = plt.figure()

  for i, (inputs, labels) in enumerate(val_dataloader):
    if bool(random.getrandbits(1)):
      continue

    inputs = inputs.to(device)
    labels = labels.to(device)

    outputs = model(inputs)
    _, preds = torch.max(outputs, 1)

    for j in range(inputs.size()[0]):
      images_so_far = images_so_far+1
      ax = plt.subplot(num_images//2, 2, images_so_far)
      ax.axis("off")
      ax.set_title("Prediccted: {}, label: {}".format(class_names[preds[j]], class_names[labels[j]]))

      imgshow(inputs.cpu().data[j],mean,std)

      if images_so_far == num_images:
        return


model.to(device)
vizualize_model(model)

In [ ]:
# Test function ------------------------------------------------------------------------------------------------------
newModel=CarTypeRecognizer.load_from_checkpoint('/content/drive/MyDrive/modevSampler.ckpt',num_target_classes=4)

In [ ]:
loader = transforms.Compose([
        transforms.RandomResizedCrop(299),
        transforms.RandomHorizontalFlip(),
        transforms.RandomVerticalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(mean, std)
    ])
from PIL import Image
from torch.autograd import Variable
def image_loader(image_name):
    """load image, returns cuda tensor"""
    image = Image.open(image_name)
    image = loader(image).float()
    image = Variable(image, requires_grad=True)
    image = image.unsqueeze(0)
    return image.cuda()

In [ ]:
imgpath="/content/drive/MyDrive/Colab Notebooks/dataset/img/van.jpg"
myImg= image_loader(imgpath)
newModel.eval()
device = torch.device("cuda")
newModel.to(device)
outputs = newModel(myImg)
_, preds = torch.max(outputs, 1)
preds